# Natural Language Processing Demystified | Recurrent Neural Networks
https://nlpdemystified.org<br>
https://github.com/nitinpunjabi/nlp-demystified<br><br>


Course module for this demo: https://www.nlpdemystified.org/course/recurrent-neural-networks

**IMPORTANT**<br>
Enable **GPU acceleration** by going to *Runtime > Change Runtime Type*. Keep in mind that, on certain tiers, you're not guaranteed GPU access depending on usage history and current load.
<br><br>
Also, if you're running this in the cloud rather than a local Jupyter server on your machine, then the notebook will *timeout* after a period of inactivity.
<br><br>
Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

In [1]:
import nltk
import numpy as np
import requests
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from nltk.corpus import treebank, brown, conll2000
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Part-of-Speech Tagging with a Bidirectional LSTM

It's difficult to find free sequence labelling datasets because they're so labour-intensive to create.
<br><br>
Fortunately, **Natural Language Toolkit (NLTK)** includes enough free sets of labelled corpora for our purposes. NLTK also provides them in a convenient uniform format.<br>
https://www.nltk.org/index.html<br>
https://www.nltk.org/nltk_data/<br>
<br>
We'll use the Treebank, Brown, and CONLL-2000 datasets. 

In [2]:
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.


True

In their original form, the datasets use different part-of-speech (PoS) tag sets. We need to ensure they all use the same tagset, so we'll download a simplified set called the *universal_tagset* from NLTK.<br>

See Section 2.3 here for a list of tags: https://www.nltk.org/book/ch05.html

In [3]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

We'll then retrieve the tagged sentences from each dataset, taking care to specify they should use the *universal tagset* we just downloaded. We'll then combine them into one collection.

In [4]:
# Download all PoS-tagged sentences and place them in one list.
tagged_sentences = treebank.tagged_sents(tagset='universal') +\
                   brown.tagged_sents(tagset='universal') +\
                   conll2000.tagged_sents(tagset='universal')

print(tagged_sentences[0])
print(f"Dataset size: {len(tagged_sentences)}")

[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')]
Dataset size: 72202


Each tagged sentence is actually a list of word-tag tuples (bear in mind that NLTK's universal tagset is a reduced tagset so items such as *proper nouns* are simply tagged as *nouns*).<br>

Our model is going to take in a sequence of words, and output a sequence of PoS tags, so we need to separate the words from the tags in our dataset. The tag sequences will serve as our training labels.

In [6]:
sentences, sentence_tags = [], []

for s in tagged_sentences:
  sentence, tags = zip(*s)
  sentences.append(list(sentence))
  sentence_tags.append(list(tags))

The sentences and their respective tags are now in separate lists.

In [7]:
print(sentences[0])
print(sentence_tags[0])

['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
['NOUN', 'NOUN', '.', 'NUM', 'NOUN', 'ADJ', '.', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'NUM', '.']


In [8]:
print(len(sentences), len(sentence_tags))

72202 72202


Create train/validation/test splits. This time, we don't have a separate test set so we'll call *train_test_split* twice.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [9]:
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

x_train, x_test, y_train, y_test = train_test_split(sentences, sentence_tags, 
                                                    test_size=1 - train_ratio, 
                                                    random_state=1)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, 
                                                test_size=test_ratio/(test_ratio + validation_ratio), 
                                                random_state=1)

In [10]:
print(len(x_train), len(y_train))
print(len(x_val), len(y_val))
print(len(x_test), len(y_test))

54151 54151
10830 10830
7221 7221


Now that we have our datasets preprocessed, the next step is to vectorize. If you watched the demo on **Word Vectors**, the next few steps should look familiar.<br>
https://www.nlpdemystified.org/course/word-vectors<br>
https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_word_vectors.ipynb

First, we need to create a tokenizer for the sentences and *fit* it to the training dataset to create a vocabulary. We'll just use the default tokenizer settings which applies some light filtering, lowers the case, and separates on spaces. We'll also supply an out-of-vocabulary token (\<OOV\>) in case the tokenizer encounters words during testing/inference which it doesn't during training.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [11]:
sentence_tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<OOV>')

In [12]:
sentence_tokenizer.fit_on_texts(x_train)

In [13]:
print(f"Vocabulary size: {len(sentence_tokenizer.word_index)}")

Vocabulary size: 52041


We also need to create *another* tokenizer for the tags since our labels are also sequences. This time, we won't need an OOV token because there are only a handful of tags and, in this case, they'll all be encountered during training.

In [14]:
tag_tokenizer = keras.preprocessing.text.Tokenizer()
tag_tokenizer.fit_on_texts(y_train)

In [15]:
print(f"Number of PoS tags: {len(tag_tokenizer.word_index)}\n")
tag_tokenizer.get_config()

Number of PoS tags: 12



{'num_words': None,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': None,
 'document_count': 54151,
 'word_counts': '{"det": 126968, "verb": 174593, "adj": 80523, "adp": 136453, "noun": 286676, "adv": 51205, ".": 142935, "pron": 44684, "conj": 35060, "num": 21461, "prt": 31229, "x": 6090}',
 'word_docs': '{".": 53332, "noun": 51171, "adv": 29531, "verb": 50837, "det": 44747, "adp": 43855, "adj": 36344, "num": 11964, "conj": 24383, "pron": 26965, "prt": 21777, "x": 2682}',
 'index_docs': '{"3": 53332, "1": 51171, "7": 29531, "2": 50837, "5": 44747, "4": 43855, "6": 36344, "11": 11964, "9": 24383, "8": 26965, "10": 21777, "12": 2682}',
 'index_word': '{"1": "noun", "2": "verb", "3": ".", "4": "adp", "5": "det", "6": "adj", "7": "adv", "8": "pron", "9": "conj", "10": "prt", "11": "num", "12": "x"}',
 'word_index': '{"noun": 1, "verb": 2, ".": 3, "adp": 4, "det": 5, "adj": 6, "adv": 7, "pron": 8, "conj": 9, "prt": 10, "

In [16]:
# The set of universal PoS tags.
tag_tokenizer.word_index

{'noun': 1,
 'verb': 2,
 '.': 3,
 'adp': 4,
 'det': 5,
 'adj': 6,
 'adv': 7,
 'pron': 8,
 'conj': 9,
 'prt': 10,
 'num': 11,
 'x': 12}

Next, we need to vectorize our sentences and corresponding tags. As we did in the demo on [Word Vectors](https://github.com/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_word_vectors.ipynb), we'll use the tokenizer's *texts_to_sequences* method to convert each sentence to a sequence of integers where each integer maps to a particular token.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences

In [17]:
x_train_seqs = sentence_tokenizer.texts_to_sequences(x_train)

In [18]:
print(x_train_seqs[0])

[27, 86, 21, 479, 7, 2, 920, 10903, 20547, 3327, 5644, 337, 4]


We can use the *sequences_to_texts* method to convert a vectorized sentence back to its preprocessed form.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#sequences_to_texts

In [19]:
print(f"Original: {x_train[0]}")
print(f"Reconstructed: {sentence_tokenizer.sequences_to_texts([x_train_seqs[0]])}")

Original: ['This', 'may', 'be', 'due', 'to', 'the', 'heavy', 'interlobular', 'connective', 'tissue', 'barriers', 'present', '.']
Reconstructed: ['this may be due to the heavy interlobular connective tissue barriers present .']


Next, we'll vectorize the labels (i.e. sequences of PoS tags) using its respective tokenizer.

In [20]:
y_train_seqs = tag_tokenizer.texts_to_sequences(y_train)

In [21]:
tag_tokenizer.sequences_to_texts([y_train_seqs[0]])

['det verb verb adj adp det adj adj adj noun noun adv .']

Finally, we'll do the same with the validation inputs and labels.

In [22]:
x_val_seqs = sentence_tokenizer.texts_to_sequences(x_val)
y_val_seqs = tag_tokenizer.texts_to_sequences(y_val)

As we covered in the slides, **Recurrent Neural Networks** are capable of handling variable length sequences.<br><br>
Despite that, it's still best to pad or truncate sequences to a uniform length for one or both of these reasons:<br>
1. Performance. The longer a sequence, the higher the computation cost. One may want to truncate long sequences to a shorter length if that's feasible and doesn't result in too much performance loss.

2. When processing datasets in batches, each sequence *in a batch* usually has to be of uniform length.<br>

For simplicity, in this demo, we'll make *every* sequence be as long as the longest sequence. In other words, we'll determine how long the longest sequence is, then pad out the rest of the sequences to be the same length.<br>

A more optimized solution would be to make each sequence as long as the longest sequence in each *batch* to avoid unnecessary processing.

In [23]:
MAX_LENGTH = len(max(x_train_seqs, key=len))
print(f"Length of longest input sequence: {MAX_LENGTH}")

Length of longest input sequence: 161


We can pad the sequences with the *pad_sequences* method:<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [24]:
x_train_padded = keras.preprocessing.sequence.pad_sequences(x_train_seqs, padding='post', 
                                                            maxlen=MAX_LENGTH)

In [25]:
print(x_train_padded[0])

[   27    86    21   479     7     2   920 10903 20547  3327  5644   337
     4     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]


We'll do the same with the training label (PoS sequences)...

In [26]:
y_train_padded = keras.preprocessing.sequence.pad_sequences(y_train_seqs, padding='post', 
                                                            maxlen=MAX_LENGTH)

...and the validation dataset.

In [27]:
x_val_padded = keras.preprocessing.sequence.pad_sequences(x_val_seqs, padding='post', maxlen=MAX_LENGTH)
y_val_padded = keras.preprocessing.sequence.pad_sequences(y_val_seqs, padding='post', maxlen=MAX_LENGTH)

PoS tagging is a multiclass classification task done at each timestep, so we need to convert every tag for every sentence into a one-hot encoding (we'll look at an alternative approach when we build a language model later in this demo).<br>
https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical<br>

In [28]:
y_train_categoricals = keras.utils.to_categorical(y_train_padded)

The label (PoS tag sequence) for a single sentence is now a **sequence of one-hot encodings**. These will serve as our training targets.

In [29]:
# The one hot encodings for the first label.
print(y_train_categoricals[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [30]:
# One-hot encoding for a single tag.
print(y_train_categoricals[0][0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


We can determine the PoS tag from a one-hot encoding by seeing which index is set to 1, then using that to query the tag tokenizer's *index_word* dictionary.

In [31]:
idx = np.argmax(y_train_categoricals[0][0])
print(f"Index: {idx}")

print(f"Tag: {tag_tokenizer.index_word[idx]}")

Index: 5
Tag: det


We'll one-hot encode the validation labels as well.

In [32]:
y_val_categoricals = keras.utils.to_categorical(y_val_padded)

At this point, we're ready to build our model. We'll train word embeddings concurrently with our model (though you can use pretrained word vectors as well). If you're unfamiliar with this idea, refer to the [Word Vectors](https://www.nlpdemystified.org/course/word-vectors) module.<br><br>
There are several new things here:<br>
1. The embedding layer has a *mask_zero* parameter. We added padding in order to make our batches the same size, but we don't want the model to make PoS predictions on padding. Setting *mask_zero* to *True* makes the layers following the embedding layer ignore padding values.<br>
https://www.tensorflow.org/guide/keras/masking_and_padding<br>
https://stackoverflow.com/questions/47485216/how-does-mask-zero-in-keras-embedding-layer-work<br><br>
2. We're using a **bidirectional LSTM**. The *Bidirectional* layer is a wrapper to which we pass an *LSTM* layer. The first parameter to the *LSTM* layer is the number of units in the cell. The second parameter, *return_sequences*, controls whether the RNN returns an output for each timestep or only the last output. Since we're doing PoS-tagging, we want an output for each timestep and so *return_sequences* is set to *True*.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM<br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional


In [33]:
# For the embedding layer. "+ 1" to account for the padding token.
num_tokens = len(sentence_tokenizer.word_index) + 1
embedding_dim = 128

# For the output layer. The number of classes corresponds to the 
# number of possible tags.
num_classes = len(tag_tokenizer.word_index) + 1

In [34]:
# The set_seed call and kernel_initializer parameters are used here to
# ensure you and I get the same results. To get random weight initializations,
# remove them.
tf.random.set_seed(0)

model = keras.Sequential()

model.add(layers.Embedding(input_dim=num_tokens, 
                           output_dim=embedding_dim, 
                           input_length=MAX_LENGTH,
                           mask_zero=True))

model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True, 
                                           kernel_initializer=tf.keras.initializers.random_normal(seed=1))))

model.add(layers.Dense(num_classes, activation='softmax', 
                       kernel_initializer=tf.keras.initializers.random_normal(seed=1)))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


A few notes about the model summary:<br>

The embedding layer **output** has three dimensions:
- Batch size (it's showing as "None" because we didn't specify it upfront. We'll do it when we call *model.fit*).
- Sequence length (the sequences are all the same length now after our padding step).
- Embedding dimension.
<br><br>

The LSTM outputs a vector *twice* the size of what we specified because it's bidirectional. Recall from the slides that the outputs from the two LSTMs will be concatenated before going to the output layer.
<br><br>

The final layer's **output** also has three dimensions:
- Batch size
- Sequence length
- Output dimension (the number of possible tags).

The output will be a **sequence of probability distributions** for each input sequence. One probability distribution per tag.



In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 161, 128)          6661376   
                                                                 
 bidirectional (Bidirectiona  (None, 161, 256)         263168    
 l)                                                              
                                                                 
 dense (Dense)               (None, 161, 13)           3341      
                                                                 
Total params: 6,927,885
Trainable params: 6,927,885
Non-trainable params: 0
_________________________________________________________________


As we did in previous demos, we'll use *early stopping* with some *patience* to halt training once validation loss stops improving.<br>

The model will compare its output (sequences of softmax-generated probability distributions) against the one-hot encoded targets.



In [36]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train_padded, y_train_categoricals, epochs=20, 
                    batch_size=256, validation_data=(x_val_padded, y_val_categoricals), 
                    callbacks=[es_callback])

Epoch 1/20
212/212 [==============================] - 61s 223ms/step - loss: 0.9701 - accuracy: 0.7041 - val_loss: 0.1559 - val_accuracy: 0.9512
Epoch 2/20
212/212 [==============================] - 23s 109ms/step - loss: 0.1120 - accuracy: 0.9641 - val_loss: 0.1055 - val_accuracy: 0.9648
Epoch 3/20
212/212 [==============================] - 15s 70ms/step - loss: 0.0742 - accuracy: 0.9756 - val_loss: 0.0949 - val_accuracy: 0.9674
Epoch 4/20
212/212 [==============================] - 13s 62ms/step - loss: 0.0586 - accuracy: 0.9806 - val_loss: 0.0903 - val_accuracy: 0.9693
Epoch 5/20
212/212 [==============================] - 11s 50ms/step - loss: 0.0474 - accuracy: 0.9844 - val_loss: 0.0936 - val_accuracy: 0.9689
Epoch 6/20
212/212 [==============================] - 9s 42ms/step - loss: 0.0385 - accuracy: 0.9876 - val_loss: 0.0915 - val_accuracy: 0.9703
Epoch 7/20
212/212 [==============================] - 8s 38ms/step - loss: 0.0312 - accuracy: 0.9902 - val_loss: 0.0954 - val_accuracy:

Once our model is trained, we'll vectorize and pad the testing dataset. In the case of the labels, we'll also one-hot encode them.

In [37]:
# Preprocess the test data and test the model.
x_test_seqs = sentence_tokenizer.texts_to_sequences(x_test)
x_test_padded = keras.preprocessing.sequence.pad_sequences(x_test_seqs, padding='post', maxlen=MAX_LENGTH)

y_test_seqs = tag_tokenizer.texts_to_sequences(y_test)
y_test_padded = keras.preprocessing.sequence.pad_sequences(y_test_seqs, padding='post', maxlen=MAX_LENGTH)
y_test_categoricals = keras.utils.to_categorical(y_test_padded)

In [38]:
model.evaluate(x_test_padded, y_test_categoricals)

226/226 [==============================] - 2s 8ms/step - loss: 0.1000 - accuracy: 0.9694


[0.10003454238176346, 0.9693905711174011]

We can now use our model to tag sentences.

In [39]:
samples = [
    "Brown refused to testify.",
    "Brown sofas are on sale.",
]

The function below takes a list of strings, tokenizes and pads them, then has the model tag them. Note that if a sentence is longer than MAX_LENGTH, it'll be truncated.

In [40]:
def tag_sentences(sentences):
  sentences_seqs = sentence_tokenizer.texts_to_sequences(sentences)
  sentences_padded = keras.preprocessing.sequence.pad_sequences(sentences_seqs, 
                                                                maxlen=MAX_LENGTH, 
                                                                padding='post')

  # The model returns a LIST of PROBABILITY DISTRIBUTIONS (due to the softmax)
  # for EACH sentence. There is one probability distribution for each PoS tag.
  tag_preds = model.predict(sentences_padded)

  sentence_tags = []

  # For EACH LIST of probability distributions...
  for i, preds in enumerate(tag_preds):

    # Extract the most probable tag from EACH probability distribution.
    # Note how we're extracting tags for only the non-padding tokens.
    tags_seq = [np.argmax(p) for p in preds[:len(sentences_seqs[i])]]

    # Convert the sentence and tag sequences back to their token counterparts.
    words = [sentence_tokenizer.index_word[w] for w in sentences_seqs[i]]
    tags = [tag_tokenizer.index_word[t] for t in tags_seq]
    sentence_tags.append(list(zip(words, tags)))

  return sentence_tags


In [41]:
tagged_sample_sentences = tag_sentences(samples)

1/1 [==============================] - 2s 2s/step


In [42]:
print(tagged_sample_sentences[0])

[('brown', 'noun'), ('refused', 'verb'), ('to', 'prt'), ('testify', 'verb')]


In [43]:
print(tagged_sample_sentences[1])

[('brown', 'adj'), ('sofas', 'noun'), ('are', 'verb'), ('on', 'adp'), ('sale', 'noun')]


So that's one way to build a PoS tagger. Industrial-strength taggers use a lot more data and these days, are powered by more sophisticated models which we'll learn about when we cover *transformers*.

# Language Modelling With Stacked LSTMs

We'll build a language model trained on the *Art of War* by Sun Tzu.

In [44]:
art_of_war = requests.get('https://raw.githubusercontent.com/nitinpunjabi/nlp-demystified/main/datasets/art_of_war.txt')\
                     .text

art_of_war[:300]

'1. Sun Tzŭ said: The art of war is of vital importance to the State.\n\n2. It is a matter of life and death, a road either to safety or to\nruin. Hence it is a subject of inquiry which can on no account be\nneglected.\n\n3. The art of war, then, is governed by five constant factors, to be\ntaken into accou'

The language model we'll build will be **character**-based (as opposed to word-based). That is, given a sequence of one or more characters, the model will be asked to predict the next character.<br><br>
Character-level models have the advantage of:
- Smaller prediction space. There are only a handful of characters in the English language compared to the tens of thousands of words in a typical corpus.
- Character-level models are more resilient to out-of-vocabulary (OOV) conditions and are better able to learn the lower mechanics of language (including punctuation).<br><br>

On the other hand, character-level models need to learn a sequence of characters to "make sense" of a word (e.g. the sequence of "c", "a", "t" to identify "cat" as a pattern) which can be inefficient and result in lower performance.<br><br>
RNNs can process any kind of sequence so what's shown here can easily be applied at the word level. When we cover *transformers*, we'll learn an alternative tokenization technique called **subword tokenization** that is a middle-ground between these two approaches.

We'll initialize a Keras **Tokenizer** and set the *char_level* parameter to True so that our corpus gets tokenized into characters rather than words. It'll continue lower-casing and applying its default light filtering.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [45]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [46]:
tokenizer.fit_on_texts([art_of_war])

The tokenizer's internal dictionary now maps characters rather than words...

In [47]:
tokenizer.get_config()

{'num_words': None,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
 'lower': True,
 'split': ' ',
 'char_level': True,
 'oov_token': None,
 'document_count': 1,
 'word_counts': '{"1": 179, ".": 896, " ": 9794, "s": 3081, "u": 1467, "n": 3565, "t": 4398, "z": 20, "\\u016d": 13, "a": 3475, "i": 3573, "d": 1681, ":": 48, "h": 2558, "e": 5837, "r": 2776, "o": 3548, "f": 1238, "w": 981, "v": 478, "l": 1722, "m": 1201, "p": 769, "c": 1390, "\\n": 1443, "2": 127, ",": 634, "y": 1055, "b": 708, "j": 23, "q": 55, "g": 1007, "3": 87, "k": 345, "\\u2019": 57, "4": 66, "(": 59, ")": 59, ";": 168, "5": 58, "6": 51, "_": 62, "7": 39, "8": 36, "9": 34, "0": 38, "x": 49, "\\u2014": 16, "?": 8, "!": 8, "-": 57, "\\u201c": 3, "\\u201d": 3, "\\u0153": 7, "\\u00fc": 3, "\\u2018": 1}',
 'word_docs': '{".": 1, "s": 1, "j": 1, ";": 1, " ": 1, "u": 1, "l": 1, "_": 1, ",": 1, "o": 1, "6": 1, "n": 1, "\\u2018": 1, "1": 1, "5": 1, "7": 1, "t": 1, "e": 1, "h": 1, "2": 1, "\\u201d": 1, "\\u2019": 1, "(": 1, "

...and the resulting possibility space is much smaller.

In [48]:
print(f"Tokenizer \"Vocabulary\" size: {len(tokenizer.word_index)}")

Tokenizer "Vocabulary" size: 56


As we did when building the PoS tagger, we'll vectorize the book's characters into a sequence of integers, each integer mapping to a particular character.

In [49]:
seq = tokenizer.texts_to_sequences([art_of_war])[0]

In [50]:
print(f"Text length: {len(seq)}")

Text length: 61054


In [51]:
# Sanity check.
tokenizer.sequences_to_texts([seq[:10]])

['1 .   s u n   t z ŭ']

Our training data is currently one long sequence which we'll need to segment into training examples. To do this, we'll use the **Tensorflow Data** API which makes it easy to build preprocessing pipelines by chaining operations together.<br>
https://www.tensorflow.org/guide/data<br>
https://www.tensorflow.org/api_docs/python/tf/data<br>

To use this API, we need to first convert our vectorized corpus (which is a plain Python list) into a  **Dataset** object, which we can do using the *from_tensor_slices* method. This takes our vectorized corpus and returns a stream of tensors, one tensor for each integer. We'll then be able to perform operations on this **Dataset** object to prep our data.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset

In [52]:
slices = tf.data.Dataset.from_tensor_slices(seq)
type(slices)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

Like Python **generators**, TF **Datasets** function like iterators, so we need to do things like convert them into a list or iterate over them in order to print them.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take

In [53]:
# Create a Dataset from the first ten slices and convert to a list to output to console.
list(slices.take(10))

[<tf.Tensor: shape=(), dtype=int32, numpy=27>,
 <tf.Tensor: shape=(), dtype=int32, numpy=21>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=8>,
 <tf.Tensor: shape=(), dtype=int32, numpy=13>,
 <tf.Tensor: shape=(), dtype=int32, numpy=5>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=3>,
 <tf.Tensor: shape=(), dtype=int32, numpy=47>,
 <tf.Tensor: shape=(), dtype=int32, numpy=49>]

In [54]:
# The first ten elements from the original vectorized corpus.
seq[:10]

[27, 21, 1, 8, 13, 5, 1, 3, 47, 49]

The next step is to create the training examples. To do that, we'll use the *window* method. Calling *window* on a dataset results in a sequence of datasets each containing N elements. So calling window(100) on a dataset of 1000 will result in a sequence of 10 datasets, each containing 100 elements.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#window

Here, we're creating windows of `input_timesteps + 1`. The *input_timesteps* represents our training example length. The *+1* is there to help us create the target/label for each training example. This will be clarified further below.<br><br>
In addition, we're setting *shift* to 1. This means we'll get overlapping windows shifted by 1. e.g. if the input is [1, 2, 3, 4, ...]. The first window will contain [1, 2, 3, ...], the second window will contain [2, 3, 4, ...] and so on. This is so we can have more training examples.<br><br>
Finally, we're setting *drop_remainder* to True which ensures ALL windows contain exactly N elements. i.e. once the input contains fewer than N elements, they are ignored.

In [55]:
input_timesteps = 100
window_size = input_timesteps + 1
windows = slices.window(window_size, shift=1, drop_remainder=True)

Looking at the first few windows, we can see they're all the same length and that each subsequent window is shifted over by 1. Our corpus has now been divided into segments of length `input_timesteps + 1`.

In [56]:
for w in windows.take(3):
  arr = list(w.as_numpy_iterator())
  print(len(arr), arr)

101 [27, 21, 1, 8, 13, 5, 1, 3, 47, 49, 1, 8, 7, 4, 12, 41, 1, 3, 10, 2, 1, 7, 9, 3, 1, 6, 16, 1, 20, 7, 9, 1, 4, 8, 1, 6, 16, 1, 25, 4, 3, 7, 11, 1, 4, 17, 22, 6, 9, 3, 7, 5, 15, 2, 1, 3, 6, 1, 3, 10, 2, 1, 8, 3, 7, 3, 2, 21, 14, 14, 29, 21, 1, 4, 3, 1, 4, 8, 1, 7, 1, 17, 7, 3, 3, 2, 9, 1, 6, 16, 1, 11, 4, 16, 2, 1, 7, 5, 12, 1, 12]
101 [21, 1, 8, 13, 5, 1, 3, 47, 49, 1, 8, 7, 4, 12, 41, 1, 3, 10, 2, 1, 7, 9, 3, 1, 6, 16, 1, 20, 7, 9, 1, 4, 8, 1, 6, 16, 1, 25, 4, 3, 7, 11, 1, 4, 17, 22, 6, 9, 3, 7, 5, 15, 2, 1, 3, 6, 1, 3, 10, 2, 1, 8, 3, 7, 3, 2, 21, 14, 14, 29, 21, 1, 4, 3, 1, 4, 8, 1, 7, 1, 17, 7, 3, 3, 2, 9, 1, 6, 16, 1, 11, 4, 16, 2, 1, 7, 5, 12, 1, 12, 2]
101 [1, 8, 13, 5, 1, 3, 47, 49, 1, 8, 7, 4, 12, 41, 1, 3, 10, 2, 1, 7, 9, 3, 1, 6, 16, 1, 20, 7, 9, 1, 4, 8, 1, 6, 16, 1, 25, 4, 3, 7, 11, 1, 4, 17, 22, 6, 9, 3, 7, 5, 15, 2, 1, 3, 6, 1, 3, 10, 2, 1, 8, 3, 7, 3, 2, 21, 14, 14, 29, 21, 1, 4, 3, 1, 4, 8, 1, 7, 1, 17, 7, 3, 3, 2, 9, 1, 6, 16, 1, 11, 4, 16, 2, 1, 7, 5, 12, 1, 12, 2

The *window* method returns a nested dataset of datasets (i.e. each window is a dataset containing a tensor).

In [57]:
print(windows, '\n')

for w in windows.take(2):
  print(w)

<_WindowDataset element_spec=DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([]))> 

<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>


But our model won't accept those. It'll accept only tensors, so we need to extract the tensors from each window. To do that, we'll use *flat_map* which will flatten the dataset of datasets into a single dataset of elements. But because we want to retain our segmented sequences, we'll also pass in a *batch* function to maintain the segments (otherwise, we'll just get back one large tensor representing our whole corpus).<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#flat_map<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch

In [58]:
dataset = windows.flat_map(lambda window: window.batch(window_size))

We now have a single dataset of tensors, where each tensor is `input_timesteps+1` long and shifted by 1.

In [59]:
for d in dataset.take(2):
  print(d)

tf.Tensor(
[27 21  1  8 13  5  1  3 47 49  1  8  7  4 12 41  1  3 10  2  1  7  9  3
  1  6 16  1 20  7  9  1  4  8  1  6 16  1 25  4  3  7 11  1  4 17 22  6
  9  3  7  5 15  2  1  3  6  1  3 10  2  1  8  3  7  3  2 21 14 14 29 21
  1  4  3  1  4  8  1  7  1 17  7  3  3  2  9  1  6 16  1 11  4 16  2  1
  7  5 12  1 12], shape=(101,), dtype=int32)
tf.Tensor(
[21  1  8 13  5  1  3 47 49  1  8  7  4 12 41  1  3 10  2  1  7  9  3  1
  6 16  1 20  7  9  1  4  8  1  6 16  1 25  4  3  7 11  1  4 17 22  6  9
  3  7  5 15  2  1  3  6  1  3 10  2  1  8  3  7  3  2 21 14 14 29 21  1
  4  3  1  4  8  1  7  1 17  7  3  3  2  9  1  6 16  1 11  4 16  2  1  7
  5 12  1 12  2], shape=(101,), dtype=int32)


The next step is to create batches from our dataset. To do this, we'll shuffle the dataset, then create batches.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle<br>


In [60]:
batch_size = 32

In [61]:
# The expected number of batches should be (len(seq) - input_timesteps) / batch_size
batches = dataset.shuffle(10000).batch(batch_size)

In [62]:
for b in batches.take(2):
  print(b)

tf.Tensor(
[[10  6 13 ...  2  1  2]
 [13 19 10 ... 18  8 21]
 [ 1 12  7 ...  1 22  9]
 ...
 [21 14 14 ...  2 12  1]
 [17 23  2 ...  6 13 19]
 [ 9  2  2 ... 14 35 21]], shape=(32, 101), dtype=int32)
tf.Tensor(
[[ 7  3  3 ...  1  3 10]
 [12  1  7 ...  5 12  1]
 [ 1 22  6 ...  1  6  5]
 ...
 [ 8 15  4 ...  7  5 12]
 [24  1 13 ...  2  1  9]
 [15 10  2 ...  1  7  5]], shape=(32, 101), dtype=int32)


We can now separate each example into an input sequence(x) and a corresponding label/target sequence(y).<br><br>
In the slides, we talked about **Teacher Forcing** where:<br>
1. At each timestep during training, the output is compared to a label.
2. At the next timestep, rather than feeding the model the previous output, we feed it the next character of the input sequence (i.e. what the model should've outputted).
<br><br>

This is why each sequence is of size *input_timesteps + 1*. Each sequence is now going to be separated into TWO sequences. The first sequence will be the training input and will be of length *input_timesteps* (i.e. everything but the LAST character). The second sequence will be the label/target and will consist of all the sequence elements shifted by 1 (i.e. everything but the FIRST character). If this is confusing, refer to the accompanying video for this module where we cover training language models.<br><br>

So if a sequence is "she swam in the lake", then:
- The input will be "she swam in the lak" (drop the last character)
- The target/label will be "he swam in the lake" (drop the first character)

In [63]:
xy_batches = batches.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

Each batch now consists of a set of input sequences and a corresponding set of label/target sequences, with the labels/targets shifted by 1.

In [64]:
for b in xy_batches.take(1):
  print(b)

(<tf.Tensor: shape=(32, 100), dtype=int32, numpy=
array([[22,  6,  5, ...,  8,  2, 11],
       [ 6,  1,  4, ...,  2, 11, 12],
       [ 1, 12,  2, ...,  6,  9,  1],
       ...,
       [18,  1, 20, ...,  1,  7,  5],
       [ 9, 18, 24, ...,  4,  8,  1],
       [ 2,  1, 20, ..., 23, 18,  1]], dtype=int32)>, <tf.Tensor: shape=(32, 100), dtype=int32, numpy=
array([[ 6,  5,  1, ...,  2, 11, 24],
       [ 1,  4,  8, ..., 11, 12, 14],
       [12,  2, 16, ...,  9,  1, 17],
       ...,
       [ 1, 20,  4, ...,  7,  5, 12],
       [18, 24,  1, ...,  8,  1, 22],
       [ 1, 20,  2, ..., 18,  1, 15]], dtype=int32)>)


In [65]:
# For greater clarity, this is the first input sequence from the first batch,
# and it's corresponding label/target sequence.
for b in xy_batches.take(1):
  print("x1 length: ", len(b[0][0].numpy()))
  print("x1: ", b[0][0].numpy())
  print("\n")
  print("y1 length: ", len(b[1][0].numpy()))
  print("y1: ", b[1][0].numpy())

x1 length:  100
x1:  [ 3  2  1 10  4 17 21 14 22  9  2  3  2  5 12  1  3  6  1 23  2  1 20  2
  7 26 24  1  3 10  7  3  1 10  2  1 17  7 18  1 19  9  6 20  1  7  9  9
  6 19  7  5  3 21 14 14 29 30 21  1  4 16  1 10  2  1  4  8  1  3  7 26
  4  5 19  1 10  4  8  1  2  7  8  2 24  1 19  4 25  2  1 10  4 17  1  5
  6  1  9  2]


y1 length:  100
y1:  [ 2  1 10  4 17 21 14 22  9  2  3  2  5 12  1  3  6  1 23  2  1 20  2  7
 26 24  1  3 10  7  3  1 10  2  1 17  7 18  1 19  9  6 20  1  7  9  9  6
 19  7  5  3 21 14 14 29 30 21  1  4 16  1 10  2  1  4  8  1  3  7 26  4
  5 19  1 10  4  8  1  2  7  8  2 24  1 19  4 25  2  1 10  4 17  1  5  6
  1  9  2  8]


The last step before we can build our model is to one-hot encode the **inputs**. We're doing this because:
1. We're not using embeddings for the input. We can, but since this is a character model with just a few dozen possible choices, we can get away with one-hot encoding. There's also no reason to think a particular letter should be closer to another in vector space as we would want in a word-level model.

2. Since we're not using embeddings and our input is categorical, we need to one-hot encode.

Note that despite our labels ALSO being categorical, we are NOT one-hot encoding them this time. This is because we'll be using a loss function that can help us skip that step (more below).

In [66]:
num_tokens = len(tokenizer.word_index) + 1

# One-hot encode the input sequences, don't do anything with the label/target sequences.
xy_batches = xy_batches.map(lambda inputs, labels: (tf.one_hot(inputs, depth=num_tokens), labels))

In [67]:
# Each input sequence is now a sequence of one-hot encodings.
for b in xy_batches.take(1):
  print("x1: ", b[0][0].numpy())
  print("\n")
  print("y1: ", b[1][0].numpy())

x1:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


y1:  [ 2  7  8  6  5  8 21 14 14 44 21  1  2  7  9  3 10  1 15  6 17 22  9  4
  8  2  8  1 12  4  8  3  7  5 15  2  8 24  1 19  9  2  7  3  1  7  5 12
  1  8 17  7 11 11 28  1 12  7  5 19  2  9  1  7  5 12  1  8  2 15 13  9
  4  3 18 28 14  6 22  2  5  1 19  9  6 13  5 12  1  7  5 12  1  5  7  9
  9  6 20  1]


At this point, we've:
- Segmented our corpus into fixed-length sequences.
- Created training and label/target sequences.
- Organized them into batches.

The last step is to add some **prefetching**. This is an optimization step. This way, while the model trains on the current batch of data, the pipeline reads and prepares the next batch.<br>
https://www.tensorflow.org/guide/data_performance#prefetching

In [68]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

We can now build our model. There are three new things here:
1. We're stacking two LSTMs. The sequential output of the first LSTM will become the sequential input to the second LSTM.<br><br>
2. We're adding some *recurrent_dropout*. This drops connections between the recurrent units (i.e. the dropout is applied horizontally across time). You can still use regular *dropout* as well which will be applied to the inputs/outputs. Refer to this paper for more information:<br>
https://arxiv.org/abs/1512.05287<br><br>
3. We're using **sparse_categorical_crossentropy**. This allows us to provide labels as integers for multiclass classification rather than one-hot encodings.<br>
https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class

In [69]:
model = keras.models.Sequential()

model.add(layers.LSTM(128, return_sequences=True, input_shape=[None, num_tokens], recurrent_dropout=0.2))
model.add(layers.LSTM(128, return_sequences=True, input_shape=[None, num_tokens], recurrent_dropout=0.2))

model.add(layers.Dense(num_tokens, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy", optimizer='adam')


In [70]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, None, 128)         95232     
                                                                 
 lstm_2 (LSTM)               (None, None, 128)         131584    
                                                                 
 dense_1 (Dense)             (None, None, 57)          7353      
                                                                 
Total params: 234,169
Trainable params: 234,169
Non-trainable params: 0
_________________________________________________________________


Because this model takes a few hours to train, we're using **model checkpoints** to save the weights after every epoch. This way, if something goes wrong with our system during training, we can reload the last set of weights from the checkpoint, and resume training from there.<br>
https://keras.io/api/callbacks/model_checkpoint/

In [71]:
filepath="./ArtofWarLM/training1/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 save_weights_only=True,
                                                 verbose=1)

When calling the model's *fit* method, we:<br>
1. simply pass in the batches as is (no need to separate into explicit x and y arguments).
2. pass the model checkpoint callback to save the weights after every epoch

**NOTE**: The call to *fit* below is commented out. Because this model takes a few hours to train, I trained it ahead of time and saved it. If you want to train it yourself, feel free to uncomment and execute it. Also, because of the random weight initialization, your trained model's output will likely differ from mine.

In [72]:
#
# history = model.fit(xy_batches, epochs=50, callbacks=[cp_callback])
#

Once training is complete, we can call the model's *save* method to save its weights and metadata. Again, this is commented out because I already trained the model previously.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/Model#save<br>
https://www.tensorflow.org/guide/keras/save_and_serialize

In [73]:
#
# model.save('art_of_war_char_level_lm')
#

Download and unzip the previously trained model...

In [74]:
!wget https://github.com/nitinpunjabi/nlp-demystified/raw/main/models/art_of_war_char_level_lm.zip
!unzip -o art_of_war_char_level_lm.zip

--2023-05-18 00:57:22--  https://github.com/nitinpunjabi/nlp-demystified/raw/main/models/art_of_war_char_level_lm.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nitinpunjabi/nlp-demystified/main/models/art_of_war_char_level_lm.zip [following]
--2023-05-18 00:57:23--  https://raw.githubusercontent.com/nitinpunjabi/nlp-demystified/main/models/art_of_war_char_level_lm.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691531 (2.6M) [application/zip]
Saving to: ‘art_of_war_char_level_lm.zip’

art_of_war_char_lev 100%[===================>]   2.57M  --.-KB/s    in 0.01s   

2023-05-18 00:57

...and load it.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model

In [75]:
model = keras.models.load_model('art_of_war_char_level_lm')

Now that we have a trained model, let's generate some text.<br><br>
The function below takes some seed text and uses that to generate a certain number of characters. For each character, it uses the generated text so far as the input. It's not the most efficient function but it'll work here.<br><br>
There's also a *temperature* parameter. The next character is picked from a probability distribution. By dividing the log of this distribution by *temperature*, we can influence the randomness of the output.<br><br>
When the temperature is low (< 1), the probability distribution sharpens and the model will be more strict in recreating the original text. As we raise the temperature, the distribution flattens and there's a higher chance the model picks something unexpected, resulting in greater surprise in the output. In practice, a high enough temperature will result in nonsense.

In [76]:
def generate_text(model, tokenizer, seed_text, num_chars=200, temperature=1):

  text = seed_text  

  for _ in range(num_chars):

    # Take the last *input_timesteps* number of characters in the text so far
    # as input.
    input = np.array(tokenizer.texts_to_sequences([text[-input_timesteps:]]))
    input = tf.one_hot(input, num_tokens)

    # Create probability distribution for next character adjusted by temperature.
    preds = model.predict(input)[0, -1:, :] # <-- We want only the last character so we're extracting the softmax output for that.
    preds = tf.math.log(preds) / temperature

    # Sample next character and add to running text.
    next_char = tf.random.categorical(preds, num_samples=1)
    next_char = tokenizer.sequences_to_texts(next_char.numpy())[0]

    text += next_char

  return text


In [77]:
%%time
print(generate_text(model, tokenizer, "Banana peels on the battlefield can", num_chars=300, temperature=0.2))

1/1 [==============================] - 0s 85ms/step
Banana peels on the battlefield can ever beseens the applies and and abroad,
and men will drop down exhausted on the highways. as many as seven
hundred thousand families will be impeded in their labor.

2. hostile armies may face each other for years, striving for the
victory which is decided in a single day. this being so, to remain
CPU times: user 36.9 s, sys: 874 ms, total: 37.8 s
Wall time: 40.4 s


In [78]:
print(generate_text(model, tokenizer, "It's time to release the Kraken when", num_chars=300, temperature=0.5))

1/1 [==============================] - 0s 74ms/step
It's time to release the Kraken when the moon is in the constellations of the sieve, the
wall, the wing or the cross-bar; for these four are all days of rising
wind.

5. in attacking with fire, one should be prepared to meet five possible
developments:

6. (1) when fire breaks out inside the enemy’s camp, respond at once
with an attac


In [79]:
print(generate_text(model, tokenizer, "Crush your enemies, see them driven before you, and", num_chars=300, 
                    temperature=1))

1/1 [==============================] - 0s 74ms/step
Crush your enemies, see them driven before you, and when the momen soldiers into positions whence there is no escape, and
they will prefer death tof the
yin dy, and though starting after him to come up.

11. if the enemy his tactics him where he seeps in a real
advantage, we may everyour at its tail, and
you will be attacked by its head; striving fo


In [80]:
print(generate_text(model, tokenizer, "What is best in life?", num_chars=300, temperature=2))

1/1 [==============================] - 0s 76ms/step
What is best in life?

15. nowing our own spies to know of wind, the spoilm by is local ivolverr
besousinveres wallsh and
swampes, but thren cto wordren is attack some intactics
luvarginy inninghoulds of the hou hae utill is
tupports
matement cleads.” it
is with your litfate and to wings, when you must rush in.

66. for


A few observations of the preceding outputs:
1. Despite being a character-level model, the model managed to "learn" spelling, cadence, punctuation, spacing, grammar, and even numbered bullet points just from trying to predict the next character.

2. It's pretty cool how the model manages to take our initial seed text and complete a sentence with it before moving on.

3. We can see the output getting increasingly nonsensical as the temperature rises. What temperature to use ultimately depends on the nature of your corpus and your goals with the language model.

Also, in contrast to our language model, GPT-3 has 175 billion parameters and was trained on 45 terabytes of data, but the high-level principle of learning through prediction remains the same.

# Further Exploration
1. Check out *Sunspring*, a sci-fi short written by an LSTM. The director and actors played the script straight and the result is hilarious.<br>
https://www.youtube.com/watch?v=LY7x2Ihqjmc<br>
https://en.wikipedia.org/wiki/Sunspring<br><br>
2. Everything we learned here can be applied at the word-level. Try creating a word-level language model with a different corpus (maybe download something from https://www.gutenberg.org/) and try using word embeddings.<br><br>
3. We didn't evaluate our language model using perplexity. Find out online how to do it.